<a href="https://colab.research.google.com/github/Tukumam/aps_2_otimizacao_de_sistema/blob/main/apsOtimizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CARTEIRA DE INVESTIMENTOS**
Disciplina: Otimização de Sistemas

Turma: 151 R

Aluno: Claudio M Garcia Pinto

Matrícula: 2016200038

# Uma Abordagem de Otimização

# 1  Importando bibliotecas

In [212]:
# Confiuração do yahoo finance

import pandas as pd
import numpy as np
import pandas_datareader.data as web
import pandas_datareader.data as pdr

!pip install -U matplotlib
import matplotlib.pyplot as plt

!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf

#Sobrescrevendo função de busca
yf.pdr_override()

import seaborn as sns
sns.set()

import random

!pip install riskfolio-lib
import riskfolio as rp


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 2 - Importando e tratando dados

In [213]:
tickers_ibovespa= "ABEV3.SA AZUL4.SA B3SA3.SA BBAS3.SA BBDC3.SA BBDC4.SA BBSE3.SA BPAC11.SA BRAP4.SA BRFS3.SA BRKM5.SA BRML3.SA CCRO3.SA CIEL3.SA CMIG4.SA COGN3.SA CRFB3.SA CSAN3.SA CSNA3.SA CVCB3.SA CYRE3.SA ECOR3.SA EGIE3.SA ELET3.SA ELET6.SA EMBR3.SA ENBR3.SA EQTL3.SA FLRY3.SA GGBR4.SA GOAU4.SA GOLL4.SA HAPV3.SA HYPE3.SA IRBR3.SA ITSA4.SA ITUB4.SA JBSS3.SA KLBN11.SA LREN3.SA MGLU3.SA MRFG3.SA MRVE3.SA MULT3.SA NTCO3.SA PETR3.SA PETR4.SA QUAL3.SA RADL3.SA RAIL3.SA RENT3.SA SANB11.SA SBSP3.SA SULA11.SA SUZB3.SA TAEE11.SA TOTS3.SA UGPA3.SA USIM5.SA VALE3.SA WEGE3.SA YDUQ3.SA"

#Baixando preço de fechamento ajustado
dados_do_yahoo = yf.download(tickers=tickers_ibovespa, period='1y')["Adj Close"]

#Baixando 1 ano de cotações do BOVA!!
ibovespa = yf.download('BOVA11.SA', period='1y')["Adj Close"]
ibovespa = ibovespa / ibovespa.iloc[0]

dados_do_yahoo.dropna(how='all', inplace=True)
dados_do_yahoo.dropna(axis=1, inplace=True, thresh=251)
dados_do_yahoo




[*********************100%***********************]  62 of 62 completed
[*********************100%***********************]  1 of 1 completed


,ABEV3.SA,AZUL4.SA,B3SA3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BPAC11.SA,BRAP4.SA,BRFS3.SA,...,SBSP3.SA,SULA11.SA,SUZB3.SA,TAEE11.SA,TOTS3.SA,UGPA3.SA,USIM5.SA,VALE3.SA,WEGE3.SA,YDUQ3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2021-11-01,1670.7947%,2710.0000%,1142.7019%,2712.0424%,1535.5392%,1806.9860%,2048.1060%,2303.4395%,3746.3943%,2310.0000%,...,3500.2510%,2573.3252%,4761.2953%,3269.3462%,3282.8678%,1247.6077%,1227.3540%,6621.0815%,3633.1680%,2103.0672%
2021-11-03,1716.0292%,2770.0001%,1144.6096%,2729.7195%,1531.7949%,1791.9630%,2100.1213%,2325.8978%,3463.9160%,2383.0000%,...,3585.4797%,2723.9977%,4845.0150%,3335.0887%,3403.8681%,1290.7278%,1158.9594%,6118.4574%,3703.9444%,2246.9769%
2021-11-04,1720.8414%,2623.0000%,1138.8865%,2665.5235%,1448.5928%,1673.3768%,2102.9079%,2283.9106%,3442.1871%,2328.0001%,...,3507.1087%,2727.9629%,4799.2611%,3282.6725%,3416.7618%,1186.2814%,1152.4011%,6048.8773%,3707.8766%,2182.4656%
2021-11-05,1722.7663%,2840.0000%,1179.9017%,2725.0675%,1507.2724%,1755.9479%,2121.4848%,2386.4374%,3364.2620%,2370.9999%,...,3543.3552%,2789.4211%,4804.1286%,3266.6813%,3471.3108%,1193.9472%,1128.9783%,5869.4344%,3703.9444%,2231.0970%
2021-11-08,1707.3673%,2815.0000%,1167.5018%,2742.7444%,1477.4944%,1715.5407%,2119.6270%,2325.8978%,3493.1377%,2317.0000%,...,3490.4549%,2795.3690%,4830.4123%,3236.4750%,3421.7205%,1193.9472%,1180.5084%,6188.9530%,3646.9303%,2266.8264%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-26,1510.0000%,1429.0000%,1318.0000%,3811.0001%,1580.0000%,1887.0001%,2804.0001%,2632.0000%,2552.0000%,1195.0000%,...,5684.9998%,2071.9999%,5279.9999%,3927.9999%,2942.0000%,1221.0000%,745.0000%,7352.9999%,3798.0000%,1338.0000%
2022-10-27,1556.0000%,1464.0000%,1388.0000%,3859.0000%,1611.0001%,1921.9999%,2918.0000%,2729.0001%,2468.0000%,1233.0000%,...,5813.0001%,2160.0000%,5311.9999%,3963.0001%,3076.0000%,1289.0000%,767.0000%,7091.0004%,3883.0002%,1487.0000%
2022-10-28,1577.0000%,1481.0000%,1418.0000%,3882.0000%,1623.0000%,1933.0000%,2932.0000%,2742.0000%,2386.0001%,1235.0000%,...,5954.0001%,2254.9999%,5511.0001%,3952.9999%,3171.9999%,1299.0000%,735.0000%,6744.9997%,3847.0001%,1530.0000%


In [214]:
#Variação percentual dos Ativos
retorno = dados_do_yahoo.pct_change()
retorno.dropna(how='all', inplace=True)

retorno

,ABEV3.SA,AZUL4.SA,B3SA3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BPAC11.SA,BRAP4.SA,BRFS3.SA,...,SBSP3.SA,SULA11.SA,SUZB3.SA,TAEE11.SA,TOTS3.SA,UGPA3.SA,USIM5.SA,VALE3.SA,WEGE3.SA,YDUQ3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2021-11-03,2.7074%,2.2140%,0.1669%,0.6518%,-0.2438%,-0.8314%,2.5397%,0.9750%,-7.5400%,3.1602%,...,2.4349%,5.8552%,1.7583%,2.0109%,3.6858%,3.4562%,-5.5725%,-7.5913%,1.9481%,6.8428%
2021-11-04,0.2804%,-5.3069%,-0.5000%,-2.3517%,-5.4317%,-6.6177%,0.1327%,-1.8052%,-0.6273%,-2.3080%,...,-2.1858%,0.1456%,-0.9443%,-1.5717%,0.3788%,-8.0921%,-0.5659%,-1.1372%,0.1062%,-2.8710%
2021-11-05,0.1119%,8.2730%,3.6013%,2.2339%,4.0508%,4.9344%,0.8834%,4.4891%,-2.2638%,1.8471%,...,1.0335%,2.2529%,0.1014%,-0.4871%,1.5965%,0.6462%,-2.0325%,-2.9666%,-0.1060%,2.2283%
2021-11-08,-0.8939%,-0.8803%,-1.0509%,0.6487%,-1.9756%,-2.3012%,-0.0876%,-2.5368%,3.8307%,-2.2775%,...,-1.4929%,0.2132%,0.5471%,-0.9247%,-1.4286%,0.0000%,4.5643%,5.4438%,-1.5393%,1.6014%
2021-11-09,0.8455%,-1.3144%,3.2680%,-0.1357%,-0.7706%,-1.1265%,2.7169%,-4.7439%,-1.7589%,1.5969%,...,1.6559%,1.4894%,-1.2495%,1.3176%,1.7681%,4.4944%,1.2698%,-2.4556%,-0.8356%,-0.2627%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-26,1.2743%,-5.6766%,-4.8375%,-3.5434%,-3.8929%,-4.3588%,-0.7082%,-4.5685%,1.3905%,-2.3693%,...,-1.8643%,-4.0741%,1.4799%,-2.1181%,-2.1291%,-3.1721%,-2.3591%,2.2244%,8.3595%,1.7491%
2022-10-27,3.0464%,2.4493%,5.3111%,1.2595%,1.9620%,1.8548%,4.0656%,3.6854%,-3.2915%,3.1799%,...,2.2515%,4.2471%,0.6061%,0.8910%,4.5547%,5.5692%,2.9530%,-3.5632%,2.2380%,11.1360%
2022-10-28,1.3496%,1.1612%,2.1614%,0.5960%,0.7449%,0.5723%,0.4798%,0.4764%,-3.3225%,0.1622%,...,2.4256%,4.3981%,3.7462%,-0.2523%,3.1209%,0.7758%,-4.1721%,-4.8794%,-0.9271%,2.8917%


In [215]:
#Retorno Acumulado
retorno_acumulado = (1 + retorno).cumprod()
retorno_acumulado.iloc[0] = 1

retorno_acumulado

,ABEV3.SA,AZUL4.SA,B3SA3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BPAC11.SA,BRAP4.SA,BRFS3.SA,...,SBSP3.SA,SULA11.SA,SUZB3.SA,TAEE11.SA,TOTS3.SA,UGPA3.SA,USIM5.SA,VALE3.SA,WEGE3.SA,YDUQ3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2021-11-03,100.0000%,100.0000%,100.0000%,100.0000%,100.0000%,100.0000%,100.0000%,100.0000%,100.0000%,100.0000%,...,100.0000%,100.0000%,100.0000%,100.0000%,100.0000%,100.0000%,100.0000%,100.0000%,100.0000%,100.0000%
2021-11-04,102.9954%,96.7897%,99.6661%,98.2847%,94.3377%,92.6060%,102.6757%,99.1522%,91.8800%,100.7792%,...,100.1959%,106.0093%,100.7974%,100.4076%,104.0786%,95.0845%,93.8931%,91.3578%,102.0563%,103.7754%
2021-11-05,103.1106%,104.7970%,103.2554%,100.4803%,98.1592%,97.1755%,103.5828%,103.6032%,89.8000%,102.6407%,...,101.2315%,108.3975%,100.8996%,99.9185%,105.7402%,95.6989%,91.9847%,88.6477%,101.9481%,106.0878%
2021-11-08,102.1889%,103.8745%,102.1703%,101.1321%,96.2199%,94.9393%,103.4921%,100.9750%,93.2400%,100.3030%,...,99.7201%,108.6287%,101.4516%,98.9946%,104.2296%,95.6989%,96.1832%,93.4734%,100.3788%,107.7867%
2021-11-09,103.0530%,102.5092%,105.5092%,100.9949%,95.4785%,93.8699%,106.3039%,96.1848%,91.6000%,101.9048%,...,101.3714%,110.2465%,100.1840%,100.2989%,106.0725%,100.0000%,97.4046%,91.1781%,99.5401%,107.5035%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-26,90.3762%,52.7306%,115.3407%,140.5214%,102.8955%,104.4280%,136.9070%,114.2639%,68.1188%,51.7316%,...,162.4169%,80.5184%,110.8942%,120.1463%,89.6168%,97.8673%,60.6997%,111.0544%,104.5369%,63.6214%
2022-10-27,93.1293%,54.0221%,121.4665%,142.2913%,104.9143%,106.3650%,142.4731%,118.4750%,65.8767%,53.3766%,...,166.0738%,83.9381%,111.5663%,121.2169%,93.6986%,103.3177%,62.4922%,107.0973%,106.8764%,70.7063%
2022-10-28,94.3862%,54.6494%,124.0919%,143.1394%,105.6958%,106.9737%,143.1567%,119.0394%,63.6879%,53.4632%,...,170.1021%,87.6298%,115.7458%,120.9110%,96.6228%,104.1193%,59.8849%,101.8716%,105.8856%,72.7509%


# Sorteando Opções para Carteira

Cada vez que esta célula é executada uma nova carteira é gerada

In [225]:
from pandas.io.formats.style import Subset
carteira = random.sample(list(dados_do_yahoo.columns) , k=6)
carteira = retorno_acumulado.loc[: , carteira]
carteira['saldo'] = carteira.sum(axis=1)
carteira["retorno"] = carteira['saldo'].pct_change()
pd.options.display.float_format = '{:.4%}'.format

carteira.dropna(subset=['retorno'], inplace=True)

carteira



,RAIL3.SA,MRVE3.SA,UGPA3.SA,EMBR3.SA,LREN3.SA,PETR4.SA,saldo,retorno
Date,,,,,,,,
2021-11-04,104.0459%,103.4840%,95.0845%,97.2235%,102.7108%,92.8571%,595.4058%,-0.7657%
2021-11-05,102.5966%,107.6271%,95.6989%,103.4931%,104.6988%,92.3214%,606.4360%,1.8525%
2021-11-08,101.2077%,106.4030%,95.6989%,102.4631%,102.7108%,93.2857%,601.7693%,-0.7695%
2021-11-09,102.2343%,110.3578%,100.0000%,104.6574%,107.0843%,95.1428%,619.4767%,2.9426%
2021-11-10,101.6908%,115.5367%,99.6160%,101.3883%,108.5422%,94.3929%,621.1668%,0.2728%
...,...,...,...,...,...,...,...,...
2022-10-26,121.9328%,91.2180%,97.8673%,57.1876%,92.7338%,196.4765%,657.4161%,-2.1859%
2022-10-27,126.7085%,96.0870%,103.3177%,57.7698%,96.0955%,197.9782%,677.9567%,3.1244%
2022-10-28,128.4012%,97.2794%,104.1193%,58.7998%,98.0649%,195.6356%,682.3001%,0.6407%


# Aplincando os Calculos

Os cálculos para análise são palicados sobre qualquer conjunto sorteado

In [ ]:
# Cria o objeto de portfolio
port = rp.Portfolio(returns=carteira)



# Esolhendo a medida de risco
rm = 'MSV'  # Desvio semipadrão

# Estima entradas do modelo (estimativas históricas)
method_mu='hist' # Método para estimar os retornos esperados com base em dados históricos.
method_cov='hist' # Método para estimar a matriz de covariância com base em dados históricos.

# Calcula os inputs do método de otimização
port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

# Estima o portfolio ótimo

model='Classic' # Modelo clássico de Markowitz
rm = 'MV' # Medida de risco: mean-variance
obj = 'MinRisk' # Função objetivo: Minimização de Risco

w = port.optimization(model = model, rm = rm, obj = obj)

# Estima a carteira que maximiza o índice de retorno ajustado ao risco
w1 = port.optimization(model='Classic', rm=rm, obj='Sharpe', rf=0.0, l=0, hist=True)

# Estima os pontos na média da fronteira eficiente - desvio padrão
ws = port.efficient_frontier(model='Classic', rm=rm, points=20, rf=0, hist=True)

# Estimar a carteira de paridade de risco para desvio padrão
w2 = port.rp_optimization(model='Classic', rm=rm, rf=0, b=None, hist=True)



# Visualização

Para ter acesso aos resultados apneas tire a # que forma o comentário.

Não consegui as plotagens, e aceito ajuda.

In [ ]:
#w1
#w2
#ws